In [1]:
import langchain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate, load_prompt
from langchain.agents import load_tools, initialize_agent
import openai
import os
import wandb
from wandb.integration.langchain import WandbTracer
import json

In [2]:
wandb.init(
    project="generate_simple_prd",
    config={
        "model": "gpt-3.5-turbo",
        "temperature": 0
    },
    entity="arihantsheth",
    name="generate_multiple_prd_1"
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: arihantsheth. Use `wandb login --relogin` to force relogin


In [ ]:
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

prompt_template = load_prompt("./prompt_templates/generate_prd_template.json")

In [4]:
prompt_input_variables = {
    "wandb": dict(
        company_name="Weights and Biases",
        company_desc="A leading AI startup specializing in experiment tracking, data versioning, hyperparameter tuning, and much more.",
        existing_feature_list="1. System and custom metrics logging\n2. Real-time experiment visualization\n3. Comparing different hyperparameter results\n4. Automated sweeps over different hyperparameters\n5. Shareability by creating reports\n6. Data version Control",
        new_feature="Model Explainability Toolkit (MXT)",
        new_feature_desc="Enables users to interpret and explain the outputs and decision-making of their AI Models."
    ),
    "langchain": dict(
        company_name="Langchain",
        company_desc="A young AI startup making it easier to use and connect Large Language Models (LLMs) to other tools such as Vector Database, Agents, Prompt Templates, and more.",
        existing_feature_list="1. Models: A suite of pretrained LLMs\n2. Prompts: Templates for prompt engineering and prompt tuning\n3. Memory: Ability to connect LLMs to vector databases\n4. Indexes: Text splitters\n5. Chains: Chain multiple prompts and LLMs together\n6. Agents: Drives decision making processes using LLMs",
        new_feature="Model Training Wizard (MTW)",
        new_feature_desc=" A user-friendly interface that guides users through the process of training their own custom language models. It would provide step-by-step instructions, dataset management tools, and performance evaluation metrics to help users create high-quality models tailored to their specific needs."
    )
}

In [5]:
prompts = dict()
for company in prompt_input_variables:
    prompts[company] = prompt_template.format(**prompt_input_variables[company])

In [9]:
with open("./company_prompts/prompt_inputs.json", "w") as f:
    json.dump(prompt_input_variables, f)

with open("./company_prompts/prompts.json", "w") as f:
    json.dump(prompts, f)

In [7]:
outputs = dict()

for company in prompts:
    print(f"Generating PRD for {company}...")
    prompt = prompts[company]
    
    try:
        output = llm(prompt, callbacks=[WandbTracer()])
    except Exception as e:
        print(e)
        output = f"ERROR: {e}"

    outputs[company] = output
    wandb.log({f"{company}_prd": output})

Generating PRD for wandb...
Generating PRD for langchain...


In [8]:
for company in outputs:
    with open(f"./generated_prds/{company}_prd.md", "w") as f:
        f.write(outputs[company])

In [10]:
wandb.finish()

langchain_prd,# Product Requiremen...
wandb_prd,# Product Requiremen...
